In [ ]:
#@title # **下載**
# 初始化
from google.colab import drive
import os, json, urllib.parse, re, requests
if not os.path.exists("/content/DL"):
  !mkdir -p /content/DL
  !rm -rf /content/sample_data
  !apt-get install aria2
  !pip install uv
  !uv pip install gigafile --system

%cd /content/DL
link = "" #@param{type:"string"}
from_direct_link = False #@param{type:"boolean"}
from_torrent = False #@param{type:"boolean"}
from_gigafile = False #@param{type:"boolean"}
from_pixeldrain = False #@param{type:"boolean"}
from_gofile = False #@param{type:"boolean"}

if from_torrent:
  !aria2c --summary-interval=10 --seed-time=0 --allow-overwrite=true "{link}"

if from_direct_link:
  !aria2c --summary-interval=10 -c -x 16 -k 1M -s 16 "{link}"

if from_gigafile:
  !gfile download "{link}"

if from_pixeldrain:
  url_check = re.match('https://pixeldrain.com/u/', link)
  if url_check:
    pd_id = link[url_check.span()[1]: len(link)]
    bypass = 'https://pd.1drv.eu.org/' + pd_id
    !aria2c --summary-interval=10 -c -x 16 -k 1M -s 16 "{bypass}"
  else:
    print("URL格式錯誤！")
    print("正確格式應為`https://pixeldrain.com/u/XXXXXXXX`")

if from_gofile:
  if not os.path.exists("/content/gofile-downloader"):
    %cd /content
    # 安裝gofile downloader
    # https://github.com/ltsdw/gofile-downloader
    !git clone https://github.com/ltsdw/gofile-downloader.git
    %cd gofile-downloader
    !uv pip install -r requirements.txt --system
    %cd /content/DL
  !python /content/gofile-downloader/gofile-downloader.py "{link}"


print("\nDone!")

---
# **上傳**

In [ ]:
#@title # **上傳到Buzzheavier**

filename = "" #@param{type:"string"}

BuzzUpUrl = "https://w.buzzheavier.com/" + urllib.parse.quote(filename)
BuzzUp = !curl -#o - -T "$filename" "$BuzzUpUrl" | cat
try:
  if int(json.loads(BuzzUp[-1]).get('code')) == 201:
    print("下載網址： https://buzzheavier.com/", json.loads(BuzzUp[-1]).get('data').get('id'), sep='')
except:
  print("下面data中的id放在 `https://buzzheavier.com/\"這裡\"` 就是下載網址\n如果沒有id那就是出錯啦\n", BuzzUp[-1])

In [ ]:
#@title # **上傳到Pixeldrain**
#@markdown **Pixeldrain上限 [20](https://pixeldrain.com/) GB**

#@markdown 現在需要帳號ㄌ！

filename = "" #@param{type:"string"}
pd_api_key = "" #@param{type:"string"}

if pd_api_key == "":
  print("請填入API密鑰！")
  print("https://pixeldrain.com/user/api_keys")
else:
  pdupload = !curl -T "{filename}" -u :$pd_api_key https://pixeldrain.com/api/file/
  res = json.loads(pdupload[0])
  print("下載網址： https://pixeldrain.com/u/", res.get('id'), sep='')
  print("Bypass: https://pd.1drv.eu.org/", res.get('id'), sep='')

In [ ]:
#@title # **上傳到Gofile**

filename = "" #@param{type:"string"}
# 就是顯示更詳細的資料
dev_mode = False #@param{type:"boolean"}

# !curl -X GET 'https://api.gofile.io/servers'
server = !curl -X GET 'https://api.gofile.io/servers'
good_server = json.loads(server[0]).get('data').get('servers')[0].get('name')
go_url = f"https://{good_server}.gofile.io/contents/uploadfile"
# !curl -X POST https://store1.gofile.io/contents/uploadfile -F "file=@{filename}"
go_up = !curl -X POST $go_url -F "file=@{filename}"
print('下載網址：', json.loads(go_up[0]).get('data').get('downloadPage'))

if dev_mode:
  gofile_token = json.loads(go_up[0]).get('data').get('guestToken')
  parentID = json.loads(go_up[0]).get('data').get('parentFolder')
  print('guestToken： ', gofile_token)
  print('parentFolder: ', parentID)
  print('server: ', good_server)
  # !curl -H "Authorization: Bearer " -X POST https://store.gofile.io/contents/uploadfile -F "file=@" -F "folderId="

In [ ]:
#@title # **上傳到Gigafile**

filename = "" #@param{type:"string"}

!gfile -p -n 24 upload "{filename}"

In [ ]:
#@title # **上傳到ViKiNG FiLE**
import os

filename = "" #@param{type:"string"}

if not os.path.exists("/content/venv"):
  %cd /content
  # viking-file套件要3.13以上，只能這樣搞
  !uv python install 3.13 -i /content
  !uv venv venv -p 3.13
  !uv pip install viking-file -p venv/bin/python

!rm upload_script.py
!wget https://raw.githubusercontent.com/chikenscrach/ColabDL/refs/heads/main/upload_script.py

# 處理檔案位置
# !sed -i "s#file_path = Path(\"\")#file_path = Path(\"{filename}\")#" upload_script.py

try:
  with open("upload_script.py", 'r', encoding='utf-8') as f:
    lines = f.readlines()

  new_lines = []
  found = False
  for line in lines:
    if 'file_path = Path("")' in line:
      new_line = f'file_path = Path({repr(filename)})\n'
      new_lines.append(new_line)
      found = True
    else:
      new_lines.append(line)

  if found:
    with open("upload_script.py", 'w', encoding='utf-8') as f:
      f.writelines(new_lines)
    !grep "file_path" upload_script.py
  else:
    print("有問題ㄏㄏ")

except FileNotFoundError:
  print("沒下載到upload_script.py")

!/content/venv/bin/python /content/upload_script.py

In [ ]:
#@title # 上傳到Google Drive
#@markdown **資料夾位置記得加 "/"**

google_drive_path = "ColabDownloads/" #@param {type:"string"}

if not os.path.exists('/content/gdrive'):
  drive.mount('/content/gdrive')

!rsync -avh --progress . "/content/gdrive/MyDrive/"{output_path}

print("\033[32mDone!\033[0m")

In [ ]:
#@title # **上傳到catbox**
#@markdown **catbox上限 [200](https://catbox.moe/) MB**

filename = "" #@param{type:"string"}

!curl -F "reqtype=fileupload" -F "fileToUpload=@{filename}" https://catbox.moe/user/api.php

---
# 其他

In [ ]:
#@title # 👇 **移除下載檔案 (清理一下colab)**
!rm -rf /content/DL/*
%cd /content/DL
print("乾淨ㄌ")

In [ ]:
# 自行發揮

---
# 以下小工具


In [ ]:
#@title # Colab終端機
#@markdown [colab-xterm](https://github.com/InfuseAI/colab-xterm)
!pip install colab-xterm
%load_ext colabxterm
%xterm

In [ ]:
#@title # Telegram 工具
#@markdown [tdl](https://github.com/iyear/tdl) [文件](https://docs.iyear.me/tdl/)
!curl -sSL https://docs.iyear.me/tdl/install.sh | sudo bash
# upload tdl
# unzip tdl.zip -d ~/.tdl

In [ ]:
#@title # Mega帳號產生器
#@markdown 使用 [MEGAcmd](https://github.com/meganz/MEGAcmd) 和 [mail.tm](https://mail.tm/) API 產生帳號
# 使用方法 https://github.com/meganz/MEGAcmd/blob/master/UserGuide.md
import random, string, json, threading, re


def generate_account(domain, username_length=8, password_length=12):
    username = ''.join(random.choices(string.ascii_lowercase + string.digits, k=username_length))
    password_characters = string.ascii_letters + string.digits + "!@#$%^&*()"
    password = ''.join(random.choices(password_characters, k=password_length))
    address = f"{username}@{domain}"
    return {"address": address, "password": password}


def wtfmega():
  # 防止卡住，我想不到更好的方法，先這樣處理
  !mega-ls


%cd /content
# 安裝 MEGAcmd
# 目前我看colab的系統是Ubuntu 22.04 amd64
# 如果之後有需要做更改可以看一下系統資訊
# !cat /etc/os-release
# !uname -m
!wget https://mega.nz/linux/repo/xUbuntu_22.04/amd64/megacmd-xUbuntu_22.04_amd64.deb && sudo apt install "$PWD/megacmd-xUbuntu_22.04_amd64.deb"
t = threading.Thread(target=wtfmega)
t.start()

domain = !curl -X GET https://api.mail.tm/domains | jq -r '."hydra:member"[0].domain'
account_info = generate_account(domain[-1])
json_data = json.dumps(account_info)
curl_cmd = f'''curl -X POST https://api.mail.tm/accounts -H "Content-Type: application/json" -d '{json_data}' '''
!$curl_cmd
curl_cmd = f'''curl -X POST https://api.mail.tm/token -H "Content-Type: application/json" -d '{json_data}' '''
tmtoken = !$curl_cmd | jq -r .token
!mega-signup "{account_info.get('address')}" "{account_info.get('password')}"
maildl = !curl -H "Authorization: Bearer {tmtoken[-1]}" -X GET https://api.mail.tm/messages -H "Content-Type: application/json" | jq -r '."hydra:member"[0].downloadUrl'
!curl -H "Authorization: Bearer {tmtoken[-1]}" -X GET https://api.mail.tm"{maildl[-1]}" -o mail.txt
with open('mail.txt', 'r') as file:
    content = file.read()

confirm_link = re.search(r'https://mega\.nz/#confirm\S+', content)

if confirm_link:
    confirm_url = confirm_link.group()
else:
    print("找不到確認連結")
!mega-confirm $confirm_url "{account_info.get('address')}" "{account_info.get('password')}"
!mega-login "{account_info.get('address')}" "{account_info.get('password')}"
print(f"\n\n\n\n\nMega帳號： {account_info.get('address')}\nMega密碼： {account_info.get('password')}\n\n\n\n\n")
# 如果沒有意外可以使用：
# 上傳 !mega-put 檔案
# 下載 !mega-get link
# 分享(需輸入Yes) !mega-export -a 檔案 或 !echo Yes | mega-export -a 檔案

In [ ]:
#@title # Gdrive
#@markdown [gdrive](https://github.com/glotlabs/gdrive)
%cd /content
!wget https://github.com/glotlabs/gdrive/releases/download/3.9.1/gdrive_linux-x64.tar.gz
!tar zxvf gdrive_linux-x64.tar.gz

# 如何登入自己的帳號？請看以下網址
# https://github.com/glotlabs/gdrive/blob/main/docs/create_google_api_credentials.md

# !./gdrive account import account.tar
# !rm gdrive_linux-x64.tar.gz account.tar
%cd /content/DL
# 下載
# !/content/gdrive files download "id" --recursive
# https://drive.google.com/file/d/"這裡面的就是ID"/view
# 上傳
# !/content/gdrive files upload [OPTIONS] [FILE_PATH]

In [ ]:
#@title # YouTube影片下載
#@markdown [po_token](https://github.com/Brainicism/bgutil-ytdlp-pot-provider)

import os
yt_url = "https://youtu.be/dQw4w9WgXcQ" #@param {type:"string"}

if not os.path.exists("/root/bgutil-ytdlp-pot-provider"):
  !pip install uv
  !uv pip install yt-dlp
  !uv pip install -U bgutil-ytdlp-pot-provider
  %cd ~
  !git clone --single-branch --branch 1.2.2 https://github.com/Brainicism/bgutil-ytdlp-pot-provider.git
  %cd bgutil-ytdlp-pot-provider/server/
  !npm install
  !npx tsc
  %cd /content

!yt-dlp --ignore-errors --output "%(title)s.%(ext)s" --verbose --merge-output-format mp4 --embed-thumbnail --embed-metadata --embed-info-json --extractor-args "youtubepot-bgutilscript:script_path=/root/bgutil-ytdlp-pot-provider/server/build/generate_once.js" "{yt_url}"

In [ ]:
#@title # YouTube直播下載
#@markdown [ytarchive](https://github.com/Kethsar/ytarchive)
%cd /content
!pip install uv
!uv pip install yt-dlp
import threading
!wget https://github.com/Kethsar/ytarchive/releases/download/latest/ytarchive_linux_amd64.zip
!unzip ytarchive_linux_amd64.zip
!rm ytarchive_linux_amd64.zip

yt_url = "" #@param{type:"string"}
mp4 = False #@param{type:"boolean"}
live_chat = False #@param{type:"boolean"}


# 下載直播聊天室
def job():
  !yt-dlp -o "[%(upload_date)s] %(title)s [%(channel)s] (%(id)s)" --write-sub --sub-lang live_chat --skip-download -v --wait-for-video 1 --ignore-config "{yt_url}"


if live_chat:
  t = threading.Thread(target=job)
  t.start()
  if mp4:
    !./ytarchive --add-metadata --vp9 -o "[%(upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(upload_date)s] %(title)s [%(channel)s] (%(id)s)" --wait -t --threads 4 -v --write-description --write-thumbnail "{yt_url}" best
  else:
    !./ytarchive --add-metadata --mkv --vp9 -o "[%(upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(upload_date)s] %(title)s [%(channel)s] (%(id)s)" --wait -t --threads 4 -v --write-description --write-thumbnail "{yt_url}" best
  t.join()
else:
  if mp4:
    !./ytarchive --add-metadata --vp9 -o "[%(upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(upload_date)s] %(title)s [%(channel)s] (%(id)s)" --wait -t --threads 4 -v --write-description --write-thumbnail "{yt_url}" best
  else:
    !./ytarchive --add-metadata --mkv --vp9 -o "[%(upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(upload_date)s] %(title)s [%(channel)s] (%(id)s)" --wait -t --threads 4 -v --write-description --write-thumbnail "{yt_url}" best

In [ ]:
#@title # Speedtest
#@markdown [SPEEDTEST](https://www.speedtest.net/zh-Hant/apps/cli)
!curl -s https://packagecloud.io/install/repositories/ookla/speedtest-cli/script.deb.sh | sudo bash
!apt-get install speedtest
!echo YES | speedtest